In [ ]:
import subprocess
import shlex
import os
import random

from subprocess import PIPE
from pathlib import Path

In [ ]:
CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

In [ ]:
def submit_independent_spatially_explicit_parallel(
    parallelism_mode, delta_t, step_slice, dedup_cache, # sweetspot
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025, partitions=4, memory=16,
    reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    
    config = fr"""
    #PBS -lselect={1}:ncpus={partitions}:mem={memory}gb
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00
    
    module load mpi
    module load anaconda3/personal

    mpiexec $HOME/necsim-rust/target-mpi/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: Independent(
            delta_t: {delta_t},
            step_slice: {step_slice},
            dedup_cache: {dedup_cache},
            parallelism_mode: {parallelism_mode},
        ),

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),
        
        log: "'$TMPDIR'/event_log",

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-mpi/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    
    $HOME/necsim-rust/target-replay/release/rustcoalescence replay '(
        logs: [
            "'$(echo $TMPDIR | sed "s:\[\([0-9]\+\)\]:\[\[\]\1\[\]\]:g")'/event_log/*/*"
        ],

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-replay/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    """
    
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"qsub -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [ ]:
def submit_independent_spatially_explicit_isolated(
    parallelism_mode, delta_t, step_slice, dedup_cache, # sweetspot
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025, partitions=4, memory=16,
    reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    results = []
    
    for _ in range(repeats):
        seed = random.getrandbits(64)
        
        config = fr"""
        #PBS -lselect={1}:ncpus={1}:mem={memory}gb
        #PBS -J 0-{max(partitions - 1, 0)}
        #PBS -lwalltime={walltime}:00:00

        $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
            speciation: {speciation},
            sample: {sample},
            seed: {seed},

            algorithm: Independent(
                delta_t: {delta_t},
                step_slice: {step_slice},
                dedup_cache: {dedup_cache},
                parallelism_mode: {parallelism_mode.format(partition=f"Partition(rank:'$PBS_ARRAY_INDEX',partitions:{partitions})")},
            ),

            scenario: SpatiallyExplicit(
                habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
                dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
            ),

            log: "'$EPHEMERAL'/'$(echo $PBS_JOBID | sed "s:\[[0-9]\+\]:\[\]:g")'/event_log/'$PBS_ARRAY_INDEX'",

            reporters: [
                Plugin(
                    library: "'$HOME'/necsim-rust/target-mpi/release/deps/libnecsim_plugins_common.so",
                    reporters: [{reporters}],
                ),
            ],
        )'
        """
    
        cwd = os.getcwd()
        os.chdir(output.parent)

        # Submit the simulation batch
        pbs_jobid = subprocess.run(shlex.split(
            f"qsub -N {output.name}.isolated"
        ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
        
        pbs_jobid_folder = pbs_jobid.replace("[]", "[[][]]")
        
        config = fr"""
        #PBS -lselect={1}:ncpus={1}:mem={memory}gb
        #PBS -lwalltime={walltime}:00:00
        #PBS -W depend=afterok:{pbs_jobid}

        $HOME/necsim-rust/target-replay/release/rustcoalescence replay '(
            logs: [
                "'$EPHEMERAL'/{pbs_jobid_folder}/event_log/*/*/*"
            ],

            reporters: [
                Plugin(
                    library: "'$HOME'/necsim-rust/target-replay/release/deps/libnecsim_plugins_common.so",
                    reporters: [{reporters}],
                ),
            ],
        )'
        
        rm -rf $EPHEMERAL/{pbs_jobid_folder}/event_log
        """
        
        # Submit the simulation batch
        results.append(subprocess.run(shlex.split(
            f"qsub -N {output.name}"
        ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip())

        os.chdir(cwd)
    
    return results

In [ ]:
def submit_cuda_spatially_explicit_isolated(
    parallelism_mode, delta_t, block_size, grid_size, step_slice, dedup_cache, # sweetspot
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025, memory=24, partitions=4,
    reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    ngpus = max(int(np.ceil(memory / 24)), 1) * 24
    
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    results = []
    
    for _ in range(repeats):
        seed = random.getrandbits(64)

        config = fr"""
        #PBS -lselect={1}:ncpus={4*ngpus}:mem={24*ngpus}gb:ngpus={ngpus}:gpu_type=P100
        #PBS -J 0-{max(partitions - 1, 0)}
        #PBS -lwalltime={walltime}:00:00

        $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
            speciation: {speciation},
            sample: {sample},
            seed: {seed},

            algorithm: CUDA(
                device: {0},
                ptx_jit: {str(True).lower()},
                delta_t: {delta_t},
                block_size: {block_size},
                grid_size: {grid_size},
                step_slice: {step_slice},
                dedup_cache: {dedup_cache},
                parallelism_mode: {parallelism_mode.format(partition=f"Partition(rank:'$PBS_ARRAY_INDEX',partitions:{partitions})")},
            ),

            scenario: SpatiallyExplicit(
                habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
                dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
            ),
            
            log: "'$EPHEMERAL'/'$(echo $PBS_JOBID | sed "s:\[[0-9]\+\]:\[\]:g")'/event_log/'$PBS_ARRAY_INDEX'",

            reporters: [
                Plugin(
                    library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                    reporters: [{reporters}],
                ),
            ],
        )'
        """

        cwd = os.getcwd()
        os.chdir(output.parent)

        # Submit the simulation batch
        pbs_jobid = subprocess.run(shlex.split(
            f"qsub -N {output.name}.isolated"
        ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
        
        pbs_jobid_folder = pbs_jobid.replace("[]", "[[][]]")
        
        config = fr"""
        #PBS -lselect={1}:ncpus={1}:mem={memory}gb
        #PBS -lwalltime={walltime}:00:00
        #PBS -W depend=afterok:{pbs_jobid}

        $HOME/necsim-rust/target-replay/release/rustcoalescence replay '(
            logs: [
                "'$EPHEMERAL'/{pbs_jobid_folder}/event_log/*/*/*"
            ],

            reporters: [
                Plugin(
                    library: "'$HOME'/necsim-rust/target-replay/release/deps/libnecsim_plugins_common.so",
                    reporters: [{reporters}],
                ),
            ],
        )'
        
        rm -rf $EPHEMERAL/{pbs_jobid_folder}/event_log
        """
        
        # Submit the simulation batch
        results.append(subprocess.run(shlex.split(
            f"qsub -N {output.name}"
        ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip())

        os.chdir(cwd)
    
    return results

In [ ]:
def submit_monolithic_spatially_explicit_parallel(
    algorithm, parallelism_mode,
    repeats=10, walltime=1, speciation=0.000001, sample=0.00025, partitions=4, memory=16,
    reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    
    config = fr"""
    #PBS -lselect={1}:ncpus={partitions}:mem={memory}gb
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00
    
    module load mpi
    module load anaconda3/personal

    mpiexec $HOME/necsim-rust/target-mpi/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: {algorithm}(parallelism_mode:{parallelism_mode}),

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),
        
        log: "'$TMPDIR'/event_log",

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-mpi/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    
    $HOME/necsim-rust/target-replay/release/rustcoalescence replay '(
        logs: [
            "'$(echo $TMPDIR | sed "s:\[\([0-9]\+\)\]:\[\[\]\1\[\]\]:g")'/event_log/*/*"
        ],

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-replay/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    """
    
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"qsub -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [ ]:
for partitions in [1, 2, 4, 8, 12, 16, 24, 32, 48]:
    for algorithm in ["Classical", "Gillespie", "SkippingGillespie"]:
        for parallelism_mode in [
            "OptimisticLockstep", "Lockstep", f"Averaging(delta_sync:{1.0})", #f"Optimistic(delta_sync:{20.0})",
        ]:
            submit_monolithic_spatially_explicit_parallel(
                algorithm, parallelism_mode,
                walltime=24, memory=124, repeats=10, speciation=0.000001, sample=0.025, partitions=partitions,
                output=f"{CC_PATTERN.sub('-', algorithm).lower()}-{
                    CC_PATTERN.sub('-', parallelism_mode[:parallelism_mode.find('(')]).lower()
                }/pbs.{partitions}",
            )
    
    for parallelism_mode in ["Individuals", "Landscape", f"Probabilistic(communication:{0.25})"]:
        submit_independent_spatially_explicit_parallel(
            parallelism_mode, 2.0, 10.0, f"Relative(factor: {1.0})", # delta_t, step_slice, dedup_cache
            walltime=8, memory=48, repeats=10, speciation=0.000001, sample=0.025, partitions=partitions,
            output=f"independent-{
                CC_PATTERN.sub('-', parallelism_mode[:parallelism_mode.find('(')]).lower()
            }/pbs.{partitions}",
        )
    
    for parallelism_mode in [f"IsolatedIndividuals(event_slice:{1000000})", f"IsolatedLandscape(event_slice:{1000000})"]:
        submit_independent_spatially_explicit_isolated(
            parallelism_mode, 2.0, 10.0, f"Relative(factor: {1.0})", # delta_t, step_slice, dedup_cache
            walltime=8, memory=48, repeats=10, speciation=0.000001, sample=0.025, partitions=partitions,
            output=f"independent-{
                CC_PATTERN.sub('-', parallelism_mode[:parallelism_mode.find('(')]).lower()
            }/pbs.{partitions}",
        )
    
    for parallelism_mode in [f"IsolatedIndividuals(event_slice:{event_slice})", f"IsolatedLandscape(event_slice:{event_slice})"]:
        submit_cuda_spatially_explicit(
            parallelism_mode, """delta_t, block_size, grid_size, step_slice, dedup_cache,""" # sweetspot
            walltime=8, memory=48, repeats=10, speciation=0.000001, sample=0.025,
            output=f"cuda-{
                CC_PATTERN.sub('-', parallelism_mode[:parallelism_mode.find('(')]).lower()
            }/pbs.{partitions}",
        )